# CS486 - Artificial Intelligence
## Lesson 6 - Constraint Satisfaction Problems

The path from a start node to a goal node in a search tree problem can be more important than the goal itself. For instance, the path from Arad to Bucharest *is* the solution. 

CSPs focus on the solution to a problem with less focus on how the solution was reached. Map coloring is a good example: We care less about the *path* to solution than we do about the solution itself. CSPs are a set of variables that are assigned values within a domain that must satisfy a set of **constraints**. A lot of problems can be formulated as a CSPs, which makes them popular. There are a lot of general-purpose CSP solvers available and since they are often used to find or approximate solution to known hard problems.  

Let's take a look at how we can use the AIMA library to articulate a CSP and the various methods of computing a solution. Before we get started, import the CSP library form AIMA:

In [ ]:
import os, sys

# since aima is not distributed as a package, this hack
# is necessary to add it to Python's import search path
sys.path.append(os.path.join(os.getcwd(),'aima'))

from aima.csp import *
from aima.notebook import psource

Here's the documentation for the CSP class:

In [ ]:
%pdoc CSP

# Map Coloring

So we need four things that every CSP problem needs: **variables**, **domains**, **neighbors**, and **contraints**. Let's see how we would build a solution to a trivial map coloring problem. 

Suppose we want to assign each node in a graph a color - *red, blue or green* - such that no two connected nodes share a color. The solution for the complete three-node graph, pictured below, is obvious.  Let's see how we would solve this simple case using the CSP class. 

In [ ]:
import networkx as nx
%matplotlib inline
nx.draw(nx.complete_graph(3))

The variables are the things we want to assign. In this case, the nodes. The domains are the valid assignments that can be made to each variable. Neighbors are the sets of variables impacted by each other's assignments. Finally, the constraint method or lambda returns `*True*` if two neighbors have assignments that do not conflict.

Below is a CSP instance for our simple map coloring problem.

In [ ]:
variables = ['A','B','C']

domains = {
    'A': ['R','G','B'],
    'B': ['R','G','B'],
    'C': ['R','G','B']
}

neighbors = {
    'A': ['B','C'],
    'B': ['A','C'],
    'C': ['A','B']
}

constraints = lambda A,a,B,b: a!=b
coloring = CSP(variables,domains,neighbors,constraints)

# Backtracking

So how do we actually solve the CSP? We need an algorithm for exploring possible solutions. *Backtracking* is the uninformed search for algorithm for CSPs. It sequentially checks every combination of assignments until it finds a vaid one. Consider the following [example from the AIMA website](http://aimacode.github.io/aima-javascript/6-Constraint-Satisfaction-Problems/#backtracking-search):

In [ ]:
from IPython.display import IFrame
IFrame('http://aimacode.github.io/aima-javascript/6-Constraint-Satisfaction-Problems/#backtracking-search', width=1000, height=600)

Consider the following questions before carrying on:

* What order do you think backtracking will check solutions on our CSP above? Add a print statement to check.
* How could you improve backtracking for our problem above?
* What kind of contraints does our CSP require? Implcit? Explicit? Unary? Binary? n-ary? 

In [ ]:
backtracking_search(coloring)

In [ ]:
pieces = [
    ('B', 'P', 'O', 'Y'),
    ('W', 'B', 'P', 'R'),
    ('Y', 'B', 'G', 'W'),
    ('Y', 'G', 'B', 'R')
]

variables = []
domains = {}
neighbors = {}

def constraints(A,a,B,b):
    return True
    
scramble = CSP(variables,domains,neighbors,constraints)

In [ ]:
perfect_ten_pieces = [
    (3,2,7,9),
    (7,8,4,1),
    (1,6,8,4),
    (2,6,9,3),
    (3,2,6,9),
    (4,2,7,9),
    (7,4,2,9),
    (3,4,9,8),
    (1,8,6,3)
]